In [1]:
!pip install -q torch torchvision transformers timm
!pip install -q sentencepiece


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, pipeline
from torchvision import transforms as T
from PIL import Image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize FLAVA model and tokenizer
flava_model = AutoModel.from_pretrained("facebook/flava-full").to(device).eval()
flava_tokenizer = AutoTokenizer.from_pretrained("facebook/flava-full")

# Initialize GPT-J or GPT-Neo
language_model = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B", device=0)


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 22.18 GiB of which 18.62 MiB is free. Process 1452278 has 2.94 GiB memory in use. Process 1523601 has 10.21 GiB memory in use. Process 1668049 has 9.01 GiB memory in use. Of the allocated memory 8.67 GiB is allocated by PyTorch, and 97.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
def preprocess_image(image_path, input_size=224):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([
        T.Resize((input_size, input_size)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)
    return image_tensor


In [3]:
def get_visual_embedding(image_tensor):
    with torch.no_grad():
        visual_embedding = flava_model.get_image_features(image_tensor)  # Produces visual embedding
    return visual_embedding


In [8]:
def generate_text_from_visual(embedding, initial_prompt="Analyze the graph"):
    # Prepare the prompt text with a portion of the embedding for context
    prompt = f"{initial_prompt}: {embedding.tolist()[:10]} ..."  # Limit embedding for readability

    # Generate human-interpretable insights
    response = language_model(prompt, max_length=2000, num_return_sequences=1, no_repeat_ngram_size=2)[0]['generated_text']
    return response


In [9]:
image_path = "/mnt/code/test_l1.png"  # Replace with your image path

# Preprocess the image
image_tensor = preprocess_image(image_path)

# Generate visual embedding using FLAVA
visual_embedding = get_visual_embedding(image_tensor)

# Generate human-interpretable insights
insight = generate_text_from_visual(visual_embedding)
print(f"Insight:\n{insight}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 23.97 GiB. GPU 0 has a total capacity of 22.18 GiB of which 6.88 GiB is free. Process 1452278 has 2.94 GiB memory in use. Process 1523601 has 10.21 GiB memory in use. Process 1656181 has 2.15 GiB memory in use. Of the allocated memory 1.71 GiB is allocated by PyTorch, and 142.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)